In [1]:
!pip install mediapipe

In [3]:
!pip install numpy

In [5]:
!pip install joblib

In [7]:
!pip install pillow

In [11]:
!pip install opencv-python

In [13]:
!pip install tk

In [2]:
import os
print("Model Exists:", os.path.exists("sign_language_model.pkl"))


Model Exists: False


In [10]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Open webcam
cap = cv2.VideoCapture(0)

data = []  # List to store landmarks and labels
labels = []  # List to store corresponding labels
output_file = "sign_language_dataset.csv"

print("Press 's' to save hand landmarks for the current sign, and 'q' to quit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Flip and convert to RGB
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame
    results = hands.process(rgb_frame)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Extract landmark positions
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])
            
            # Display instruction
            cv2.putText(frame, "Press 's' to save", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # Capture data on 's' key press
            if cv2.waitKey(1) & 0xFF == ord('s'):
                sign_label = input("Enter the corresponding letter (A-Z): ")
                data.append(landmarks)
                labels.append(sign_label.upper())
                print(f"Saved: {sign_label}")
    
    # Show frame
    cv2.imshow("Sign Language Data Collection", frame)
    
    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Save collected data
if data:
    df = pd.DataFrame(data)
    df['label'] = labels
    df.to_csv(output_file, index=False)
    print(f"Dataset saved as {output_file}")
else:
    print("No data collected.")


Press 's' to save hand landmarks for the current sign, and 'q' to quit.


Enter the corresponding letter (A-Z):  C


Saved: C


Enter the corresponding letter (A-Z):  Promise


Saved: Promise


Enter the corresponding letter (A-Z):  Shy


Saved: Shy
Dataset saved as sign_language_dataset.csv


In [16]:
!pip install "pillow<10"

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.4 MB/s eta 0:00:02
   ------------ --------------------------- 0.8/2.5 MB 1.5 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.3 MB/s eta 0:00:02
   -------------------- ------------------- 1.3/2.5 MB 1.4 MB/s eta 0:00:01
   ------------------------- -------------- 1.6/2.5 MB 1.4 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.4 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.5 MB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 1.4 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 10.4.0
    Uninstalling pillow-10.4.0:
      Successfully uninstalled pillow-10.4.0


In [5]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
import os
import tkinter as tk
from tkinter import Label, Frame
from tkinter import ttk  # Import ttk for modern styling
from PIL import Image, ImageTk

# Load trained model
model_path = "sign_language_model.pkl"
if not os.path.exists(model_path):
    print("Error: Model not found! Train the model first.")
    exit()

model = joblib.load(model_path)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# File to save recognized signs
output_file = "recognized_signs.txt"
with open(output_file, "w") as f:
    f.write("Recognized Sign Language Output:\n")

# Initialize webcam
cap = cv2.VideoCapture(0)
is_running = False  # To control the webcam loop

# Create UI
root = tk.Tk()
root.title("Sign Language Recognition")
root.geometry("900x650")
root.configure(bg="#121212")  # Dark theme

# Set Mars image as application logo
try:
    logo_img = Image.open("mars.png")
    logo_imgtk = ImageTk.PhotoImage(logo_img)
    root.iconphoto(False, logo_imgtk)  # Set application window icon
except FileNotFoundError:
    print("Error: 'mars.png' not found!")

# ---- Tkinter Theming (Fixed Buttons) ----
style = ttk.Style()
style.theme_use("clam")  # Ensures color styles apply correctly

# Base button style
style.configure("TButton", font=("Arial", 16), padding=10, borderwidth=2, relief="flat")

# Start Button (Green)
style.configure("Start.TButton", background="#4CAF50", foreground="white")
style.map("Start.TButton", background=[("active", "#45a049")], foreground=[("active", "white")])

# Stop Button (Red)
style.configure("Stop.TButton", background="#D32F2F", foreground="white")
style.map("Stop.TButton", background=[("active", "#b71c1c")], foreground=[("active", "white")])

# ---- UI Frames ----
header_frame = Frame(root, bg="#1E1E1E", pady=10)
header_frame.pack(fill="x")

main_frame = Frame(root, bg="#121212")
main_frame.pack(expand=True)

footer_frame = Frame(root, bg="#1E1E1E", pady=10)
footer_frame.pack(fill="x")

# ---- Logo & Title ----
try:
    logo_img_resized = logo_img.resize((100, 100), Image.LANCZOS)
    logo_imgtk_resized = ImageTk.PhotoImage(logo_img_resized)
    logo_label = Label(header_frame, image=logo_imgtk_resized, bg="#1E1E1E")
    logo_label.image = logo_imgtk_resized  # Prevent garbage collection
    logo_label.pack(side="left", padx=10)
except NameError:
    print("Logo image not loaded.")

title_label = Label(header_frame, text="Sign Language Recognition", font=("Arial", 28, "bold"), fg="white", bg="#1E1E1E")
title_label.pack(side="left", padx=20)

# ---- Video Display ----
video_label = Label(main_frame, bg="#1E1E1E")
video_label.pack(pady=10)

# ---- Recognized Sign Display ----
recognized_label = Label(main_frame, text="Detected: ?", font=("Arial", 24, "bold"), fg="#4CAF50", bg="#121212")
recognized_label.pack(pady=20)

# ---- Process Video Function ----
def process_video():
    global is_running
    if not is_running:
        return
    
    ret, frame = cap.read()
    if not ret:
        root.after(10, process_video)
        return
    
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    recognized_letter = "?"
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = [coord for lm in hand_landmarks.landmark for coord in (lm.x, lm.y, lm.z)]

            if len(landmarks) == model.n_features_in_:  # Check if correct feature length
                landmarks = np.array(landmarks).reshape(1, -1)
                recognized_letter = model.predict(landmarks)[0]
                with open(output_file, "a") as f:
                    f.write(recognized_letter + "\n")

    # Update recognized sign label
    recognized_label.config(text=f"Detected: {recognized_letter}")

    # Convert frame for Tkinter
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    img = img.resize((640, 480), Image.LANCZOS)
    imgtk = ImageTk.PhotoImage(image=img)

    # Store a reference to avoid garbage collection
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)
    
    root.after(10, process_video)  # Call function recursively

# ---- Start/Stop Functions ----
def start_recognition():
    global is_running
    if not is_running:
        is_running = True
        process_video()

def stop_recognition():
    global is_running
    is_running = False

# ---- Buttons ----
start_button = ttk.Button(footer_frame, text="Start", style="Start.TButton", command=start_recognition)
start_button.pack(side="left", padx=20, pady=10)

stop_button = ttk.Button(footer_frame, text="Stop", style="Stop.TButton", command=stop_recognition)
stop_button.pack(side="right", padx=20, pady=10)

# Run UI
root.mainloop()

# Release resources
cap.release()
cv2.destroyAllWindows()
